# Amazon SageMaker scikit-learn Bring Your Own Model
_**Hosting a Pre-Trained scikit-learn Model in Amazon SageMaker scikit-learn Container**_

---

---

## Background

Amazon SageMaker includes functionality to support a hosted notebook environment, distributed, serverless training, and real-time hosting. We think it works best when all three of these services are used together, but they can also be used independently.  Some use cases may only require hosting.  Maybe the model was trained prior to Amazon SageMaker existing, in a different service.

This notebook shows how to use a pre-trained scikit-learn model with the Amazon SageMaker scikit-learn container to quickly create a hosted endpoint for that model.

---
## Setup

Let's start by specifying:

* AWS region.
* The IAM role arn used to give learning and hosting access to your data. See the documentation for how to specify these.
* The S3 bucket that you want to use for training and model data.

In [ ]:
import os
import boto3
import re
import json
import pandas as pd
import numpy as np
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.model import SKLearnModel
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split


region = boto3.Session().region_name

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-sklearn-byo-model"

print(f"bucket: {bucket}")

## Prepare data for model inference

We load a dataset from sklearn, and will use it to invoke SageMaker Endpoint

In [ ]:
# we use the Boston housing dataset
data = load_boston()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)

# we don't train a model, so we will need only the testing data
testX = pd.DataFrame(X_test, columns=data.feature_names)

## Download a pre trained model file

Download a pretrained Scikit-Learn Random Forest model.

We used the California Housing dataset, present in Scikit-Learn: https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset to train the model.

For more details on how to train the model with Amazon SageMaker, please refer to the [Develop, Train, Optimize and Deploy Scikit-Learn Random Forest notebook](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/scikit_learn_randomforest/Sklearn_on_SageMaker_end2end.ipynb)

In [ ]:
!aws s3 cp s3://aws-ml-blog/artifacts/scikit_learn_bring_your_own_model/model.joblib .

### Compressed the model file to a GZIP tar archive 

Note that the model file name must satisfy the regular expression pattern: `^[a-zA-Z0-9](-*[a-zA-Z0-9])*;`. The model file needs to be tar-zipped. 

In [ ]:
model_file_name = "model.joblib"

In [ ]:
!tar czvf model.tar.gz $model_file_name

## Upload the pre-trained model `model.tar.gz` file to S3

In [ ]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

## Set up hosting for the model

This involves creating a SageMaker model from the model file previously uploaded to S3.

In [ ]:
model_data = "s3://{}/{}".format(bucket, key)
print(f"model data: {model_data}")

### Write the Inference Script

When using endpoints with the Sagemaker managed Scikit Learn container, we need to provide an entry point script for inference that will **at least** load the saved model.

We will now create this script and call it `inference.py` and store it at the root of a directory called `code`.


**Note:** You would modify the script below to implement your own inferencing logic.

Additional information on model loading and model serving for Scikit Learn on SageMaker can be found [SageMaker Scikit-learn Model Server documentation.](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#deploy-a-scikit-learn-model)

There are also several functions for hosting which we won't define, like `input_fn()`, `output_fn()`, and `predict_fn()`.  These will take on their default values as described [SageMaker Scikit-learn Serve a Model documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#serve-a-model)

In [ ]:
!pygmentize ./code/inference.py

### Deploy with Python SDK

Here we showcase the process of creating a model from s3 artifacts, that could be used to deploy a model that was trained in a different session or even out of SageMaker.

In [ ]:
model = SKLearnModel(
    role=role,
    model_data=model_data,
    framework_version="0.23-1",
    py_version="py3",
    source_dir="code",
    entry_point="inference.py",
)

### Create endpoint
Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-10 minutes to complete.

In [ ]:
%%time

predictor = model.deploy(instance_type="ml.t2.medium", initial_instance_count=1)

## Validate the model for use
Now you can obtain the endpoint from the client library using the result from previous operations and generate classifications from the model using that endpoint.

### Invoke with the Python SDK

Lets generate the prediction for a single datapoint. We'll pick one from the test data generated earlier.

In [ ]:
# the SKLearnPredictor does the serialization from pandas for us
predictions = predictor.predict(testX[data.feature_names])
print(predictions)

### Alternative: invoke with `boto3`

This is useful when invoking the model from external clients, e.g. Lambda Functions, or other micro-services.

In [ ]:
runtime = boto3.client("sagemaker-runtime")

#### Option 1: `csv` serialization

In [ ]:
# csv serialization
response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint,
    Body=testX[data.feature_names].to_csv(header=False, index=False).encode("utf-8"),
    ContentType="text/csv",
)

print(response["Body"].read())

#### Option 2: `npy` serialization

In [ ]:
# npy serialization
from io import BytesIO


# Serialise numpy ndarray as bytes
buffer = BytesIO()
# Assuming testX is a data frame
np.save(buffer, testX[data.feature_names].values)

response = runtime.invoke_endpoint(
    EndpointName=predictor.endpoint, Body=buffer.getvalue(), ContentType="application/x-npy"
)

print(response["Body"].read())

### (Optional) Delete the Endpoint

If you're ready to be done with this notebook, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
predictor.delete_endpoint()